## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [2]:
dir_path = "/Users/ecem/Desktop/seperate-axis"

In [3]:
y = np.load(dir_path + "/y-10sec.npy", allow_pickle= True)

In [4]:
y.shape

(2392, 2560)

In [5]:
def converter(array):
    x= []
    for i in range(array.shape[0]):
        x.append(obspy.core.trace.Trace(array[i]))
        
        x[i].stats.sampling_rate = 256
    return np.array(x)

In [6]:
y = converter(y)


## comparison of two dataset:

Now we will to try visualize spectograms. So far we have used Obspy to visualize SCG signals. However, with obspy we are not able to plot spectograms side by side. This is important for us because we want to see spectograms of x,y and z components simultaneously for both healty and diseased people. So, from now on I will refer to those plots, for 3 components of healty and diseased (in total six plot, as "PAIR".


I have mentioned that Obpsy has a problem in side by side plotting. So I changed it source code. I have added and deleted some part to operate according to my need. ???????????

Thats why from now on we are going to use **output_spectogram** function to get values needed to plot spectogram.

These are the imports for output_spectogram function to work.

In [7]:
from obspy.imaging.cm import obspy_sequential
import math
from matplotlib import mlab
from matplotlib.colors import Normalize

In [8]:
def _nearest_pow_2(x):
    a = math.pow(2, math.ceil(np.log2(x)))
    b = math.pow(2, math.floor(np.log2(x)))
    if abs(a - x) < abs(b - x):
        return a
    else:
        return b

In [9]:
def output_spectrogram(data, samp_rate, per_lap=0.9, wlen=None, log=False,
                outfile=None, fmt=None, axes=None, dbscale=False,
                mult=8.0, cmap=obspy_sequential, zorder=None, title=None,
                show=True, clip=[0.0, 1.0]):
 
    import matplotlib.pyplot as plt
    
    # enforce float for samp_rate
    samp_rate = float(samp_rate)

    # set wlen from samp_rate if not specified otherwise
    if not wlen:
        wlen = samp_rate / 100.

    npts = len(data)
    # nfft needs to be an integer, otherwise a deprecation will be raised
    # XXX add condition for too many windows => calculation takes for ever
    nfft = int(_nearest_pow_2(wlen * samp_rate))
    if nfft > npts:
        nfft = int(_nearest_pow_2(npts / 8.0))

    if mult is not None:
        mult = int(_nearest_pow_2(mult))
        mult = mult * nfft
    nlap = int(nfft * float(per_lap))

    data = data - np.array(data).mean()
    end = npts / samp_rate

    # Here we call not plt.specgram as this already produces a plot
    # matplotlib.mlab.specgram should be faster as it computes only the
    # arrays
    # XXX mlab.specgram uses fft, would be better and faster use rfft
    specgram, freq, time = mlab.specgram(data, Fs=samp_rate, NFFT=nfft,
                                         pad_to=mult, noverlap=nlap, mode = 'psd')

    # db scale and remove zero/offset for amplitude
    if dbscale:
        specgram = 10 * np.log10(specgram[1:, :])
    else:
        specgram = np.sqrt(specgram[1:, :])
    freq = freq[1:]
    
    vmin, vmax = clip
    if vmin < 0 or vmax > 1 or vmin >= vmax:
        msg = "Invalid parameters for clip option."
        raise ValueError(msg)
    _range = float(specgram.max() - specgram.min())
    vmin = specgram.min() + vmin * _range
    vmax = specgram.min() + vmax * _range
    norm = Normalize(vmin, vmax, clip=True)

    # calculate half bin width
    halfbin_time = (time[1] - time[0]) / 2.0
    halfbin_freq = (freq[1] - freq[0]) / 2.0

    # argument None is not allowed for kwargs on matplotlib python 3.3
    kwargs = {k: v for k, v in (('cmap', cmap), ('zorder', zorder))
              if v is not None}

    if log:
        # pcolor expects one bin more at the right end
        freq = np.concatenate((freq, [freq[-1] + 2 * halfbin_freq]))
        time = np.concatenate((time, [time[-1] + 2 * halfbin_time]))
        # center bin
        time -= halfbin_time
        freq -= halfbin_freq
        # Log scaling for frequency values (y-axis)
        ax.set_yscale('log')
        # Plot times
        ax.pcolormesh(time, freq, specgram, norm=norm, **kwargs)
    else:
        # this method is much much faster!
        specgram = np.flipud(specgram)
        # center bin
        extent = (time[0] - halfbin_time, time[-1] + halfbin_time,
                  freq[0] - halfbin_freq, freq[-1] + halfbin_freq)
    return [specgram, extent, end]

    



## Plotting spectogram:

f is the frequency array, containing the frequencies of every band of the fft. Which can be used as the labels for a graph

t is the time array, containing the time at which this FFT was made relative to the source signal. Again can be used for labels.

The Sxx array contains the amplitudes and is a 2d array whose shape is the length of f by the length of t.


 The purpose of a spectogram is to take the FFT of small, equal-sized time chunks. This produces a 2D fourier transform where the X axis is the start time of the time chunk and the Y axis is the energy (or power, etc.) in each frequency in that time chunk. This allows you to see how the frequency components change over time.
 
  A spectrogram is a representation of frequency over time with the addition of amplitude as a third dimension, denoting the intensity or volume of the signal at a frequency and a time.

In [10]:
def freq_average(array):
    means= []
    for i in range(array.shape[0]):
        means.append(array[i][0:40].mean())
    return np.array(means)
        

In [11]:
output_spectrogram(y[0].data, 256)[0].shape

(2048, 40)

In [12]:
freq_average(array = output_spectrogram(y[0].data, 256)[0])

array([0.00031779, 0.0004522 , 0.00045476, ..., 0.01479482, 0.01494243,
       0.01499889])

In [13]:
freq_average(array = output_spectrogram(y[0].data, 256)[0]).shape

(2048,)

In [14]:
def spec_values(array):
    x_ = []
    for i in range(array.shape[0]):
        _x = output_spectrogram(array[i].data, 256)[0]

        x_.append(freq_average(array = _x))
        
        
    return np.array(x_)

In [15]:
type(y)

numpy.ndarray

In [16]:
y_ = spec_values(y)

In [17]:
y_[0]

array([0.00031779, 0.0004522 , 0.00045476, ..., 0.01479482, 0.01494243,
       0.01499889])

In [18]:
print(y_.shape)


(2392, 2048)


In [19]:
columns = []
for i in range(y_.shape[1]):
    columns.append("freq_y{}".format(i))
print(columns)

['freq_y0', 'freq_y1', 'freq_y2', 'freq_y3', 'freq_y4', 'freq_y5', 'freq_y6', 'freq_y7', 'freq_y8', 'freq_y9', 'freq_y10', 'freq_y11', 'freq_y12', 'freq_y13', 'freq_y14', 'freq_y15', 'freq_y16', 'freq_y17', 'freq_y18', 'freq_y19', 'freq_y20', 'freq_y21', 'freq_y22', 'freq_y23', 'freq_y24', 'freq_y25', 'freq_y26', 'freq_y27', 'freq_y28', 'freq_y29', 'freq_y30', 'freq_y31', 'freq_y32', 'freq_y33', 'freq_y34', 'freq_y35', 'freq_y36', 'freq_y37', 'freq_y38', 'freq_y39', 'freq_y40', 'freq_y41', 'freq_y42', 'freq_y43', 'freq_y44', 'freq_y45', 'freq_y46', 'freq_y47', 'freq_y48', 'freq_y49', 'freq_y50', 'freq_y51', 'freq_y52', 'freq_y53', 'freq_y54', 'freq_y55', 'freq_y56', 'freq_y57', 'freq_y58', 'freq_y59', 'freq_y60', 'freq_y61', 'freq_y62', 'freq_y63', 'freq_y64', 'freq_y65', 'freq_y66', 'freq_y67', 'freq_y68', 'freq_y69', 'freq_y70', 'freq_y71', 'freq_y72', 'freq_y73', 'freq_y74', 'freq_y75', 'freq_y76', 'freq_y77', 'freq_y78', 'freq_y79', 'freq_y80', 'freq_y81', 'freq_y82', 'freq_y83', '

In [20]:
print(len(columns))

2048


In [21]:
def dataframe_creator(array, df = pd.DataFrame()):
    values = []
    for i in range(array.shape[0]):
        values.append(array[i][0:2048].tolist())

        
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [22]:
df = dataframe_creator(y_, 
                         df =pd.DataFrame(columns = columns))

In [23]:
#df = df.reset_index()

In [23]:
df

freq_y0   freq_y1   freq_y2   freq_y3   freq_y4   freq_y5   freq_y6  \
0   0.000318  0.000452  0.000455  0.000455  0.000451  0.000445  0.000438   
0   0.000242  0.000351  0.000367  0.000384    0.0004  0.000414  0.000424   
0   0.000418  0.000595  0.000603  0.000609  0.000617  0.000621  0.000621   
0    0.00028  0.000397  0.000399  0.000398  0.000394  0.000387  0.000378   
0   0.000229   0.00033  0.000339  0.000347  0.000353  0.000357  0.000358   
..       ...       ...       ...       ...       ...       ...       ...   
0   0.000119  0.000173  0.000183  0.000194  0.000205  0.000216  0.000227   
0   0.000158  0.000228  0.000238  0.000249   0.00026  0.000268  0.000274   
0   0.000148  0.000216   0.00023  0.000245  0.000261  0.000277  0.000293   
0   0.000245  0.000348  0.000349  0.000349  0.000346  0.000342  0.000336   
0   0.000084  0.000118  0.000118  0.000117  0.000114  0.000111  0.000108   

     freq_y7   freq_y8   freq_y9  ... freq_y2038 freq_y2039 freq_y2040  \
0   0.000434  0.000433  0.000436  ...   0.011028   0.011697   0.012383   
0    0.00043  0.000432  0.000431  ...   0.007156   0.007703   0.008175   
0   0.000619  0.000615  0.000613  ...   0.006355   0.006773   0.007094   
0   0.000372  0.000368  0.000368  ...   0.009184   0.010301   0.011396   
0   0.000363  0.000375  0.000394  ...   0.009635   0.010287   0.010849   
..       ...       ...       ...  ...        ...        ...        ...   
0   0.000238  0.000249   0.00026  ...    0.01076   0.012273   0.013703   
0   0.000276  0.000275  0.000272  ...   0.018895   0.021145   0.023195   
0   0.000306  0.000317  0.000324  ...   0.021136   0.023808   0.026277   
0   0.000329  0.000322  0.000315  ...   0.017589   0.020049   0.022394   
0   0.000105  0.000102  0.000102  ...   0.004755   0.005374   0.005953   

   freq_y2041 freq_y2042 freq_y2043 freq_y2044 freq_y2045 freq_y2046  \
0    0.013054   0.013653   0.014152   0.014534   0.014795   0.014942   
0    0.008551   0.008815   0.008963   0.009001   0.008944   0.008821   
0    0.007305     0.0074   0.007381   0.007262   0.007066   0.006827   
0    0.012439   0.013396   0.014235   0.014929   0.015461   0.015824   
0    0.011287   0.011581   0.011724   0.011724   0.011605   0.011405   
..        ...        ...        ...        ...        ...        ...   
0    0.015003    0.01613   0.017053   0.017752   0.018224   0.018482   
0    0.024977   0.026438   0.027545   0.028288   0.028687    0.02879   
0    0.028463     0.0303   0.031744   0.032777   0.033408   0.033675   
0    0.024549   0.026446   0.028033   0.029274   0.030156   0.030692   
0    0.006472   0.006916   0.007271   0.007532   0.007699   0.007781   

   freq_y2047  
0    0.014999  
0    0.008683  
0    0.006595  
0    0.016018  
0    0.011188  
..        ...  
0    0.018563  
0    0.028682  
0    0.033643  
0    0.030926  
0    0.007794  

[2392 rows x 2048 columns]

In [24]:
df.to_csv(dir_path + '/y-axis-spectogram_coef.csv')